In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 18 22:04:08 2020

@author: Aron
"""

"""
Version Note
1. Add dropdown
"""

# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import sys

#import os
#import re
#import numpy as np
from flask import Flask, request

In [28]:
# Load Data ----------------

# Local DB .....
import mysql.connector

db = mysql.connector.connect(
     host="localhost",
     user="aron",
     password="57diyyLCHH4q1kwD",
     port="8889",
     database="twstock"
)

cursor = db.cursor()

In [ ]:
# PythonAnywhere .....
# import MySQLdb
#
# db=MySQLdb.connect(
#     host='aronhack.mysql.pythonanywhere-services.com',
#     user='aronhack',
#     passwd='pythonmysql2020',
#     db='aronhack$aronhack_dashboard',
#     charset = 'utf8')

In [29]:
# Load Data --------------------------

cols = ['WORK_DATE', 'SECURITY_CODE',
        'HIGH_PRICE', 'PRICE', 'LOW_PRICE']

# Historical Data .....
cursor.execute("select work_date, security_code, "
               "high, close, low "
               "from stock_data;")

historical_data = pd.DataFrame(cursor.fetchall())
historical_data.columns = cols
historical_data['TYPE'] = 'PAST'

historical_data

,WORK_DATE,SECURITY_CODE,HIGH_PRICE,PRICE,LOW_PRICE,TYPE
0,2000-01-04,1702,4.3,4.2,4.1,PAST
1,2000-01-05,1702,4.3,4.3,4.1,PAST
2,2000-01-06,1702,4.6,4.6,4.3,PAST
3,2000-01-07,1702,4.8,4.6,4.6,PAST
4,2000-01-10,1702,4.7,4.6,4.6,PAST
...,...,...,...,...,...,...
9995,2019-11-08,1707,182.8,181.8,181.3,PAST
9996,2019-11-11,1707,182.3,181.8,180.9,PAST
9997,2019-11-12,1707,182.3,181.8,180.9,PAST
9998,2019-11-13,1707,182.3,182.3,180.9,PAST


In [30]:
# Forecast Data .....
cursor.execute("select work_date, security_code, "
               "high_price, close_price, low_price "
               "from stock_forecast;")

forecast_data = pd.DataFrame(cursor.fetchall())
forecast_data.columns = cols
forecast_data['TYPE'] = 'FORECAST'

forecast_data

,WORK_DATE,SECURITY_CODE,HIGH_PRICE,PRICE,LOW_PRICE,TYPE
0,2020-05-30,0050,88.8,84.7,80.4,FORECAST
1,2020-05-31,0050,88.0,83.9,79.4,FORECAST
2,2020-06-01,0050,89.3,85.4,80.8,FORECAST
3,2020-06-02,0050,89.7,85.4,81.5,FORECAST
4,2020-06-03,0050,89.5,85.3,81.6,FORECAST
5,2020-06-04,0050,89.6,85.4,81.5,FORECAST
6,2020-06-05,0050,89.6,85.4,81.2,FORECAST
7,2020-06-06,0050,89.1,84.7,80.6,FORECAST
8,2020-06-07,0050,88.1,83.9,79.7,FORECAST
9,2020-06-08,0050,89.7,85.3,81.2,FORECAST


In [31]:
# Stock Name .....
cursor.execute("select * "
               "from stock_name;")

stock_name = pd.DataFrame(cursor.fetchall())
stock_name.columns = ['SECURITY_CODE', 'NAME']
stock_name

,SECURITY_CODE,NAME
0,0050,元大台灣50
1,0051,元大中型100
2,0052,富邦科技
3,0053,元大電子
4,0054,元大台商50
...,...,...
1112,9944,新麗
1113,9945,潤泰新
1114,9946,三發地產
1115,9955,佳龍


In [32]:
# Work Area -------------

df = pd.concat([historical_data, forecast_data])
df = df.sort_values(by=['SECURITY_CODE', 'WORK_DATE'])


df = pd.merge(df, stock_name, how='left', on=['SECURITY_CODE'])
df['SECURITY_CODE'] = df['SECURITY_CODE'] + ' ' + df['NAME']

# Add lead and lag
# https://stackoverflow.com/questions/23664877/pandas-equivalent-of-oracle-lead-lag-function

In [ ]:
# Dash ----------------------

# Unique Name
# Update, Sort by name
stock_list_pre = df['SECURITY_CODE'].unique().tolist()
stock_list = []

# Optimize
for i in stock_list_pre:
    stock_list.append({'label': i, 'value': i})


# Application ----------------------

app = dash.Dash()

colors = {
    'background': '#f5f5f5',
    'text': '#303030'
}

app.layout = html.Div(children=[
#     html.H1(
#         children='Stock Forecast',
#         style={
#             'textAlign': 'center',
#             'color': colors['text']
#         }
#     ),
#     html.Div(
#         children='Description',
#         style={
#             'textAlign': 'center',
#             'color': colors['text']
#         }
#     ),
    dcc.Dropdown(
        id='name_dropdown',
        options=stock_list,
        multi=True,
        value="MTL"
    ),
    html.Div(id="line_chart"),
    html.P('Data Source'),
    ],
    style={'backgroundColor': colors['background']}
)


@app.callback(
    Output("line_chart", "children"),
    [Input("name_dropdown", "value"),]
)


def update_output(dropdown_value):
    

    # Optimize
    data1 = [{'x': df[(df['SECURITY_CODE'] == val) & (df['TYPE'] == 'PAST')]['WORK_DATE'],
              'y': df[(df['SECURITY_CODE'] == val) & (df['TYPE'] == 'PAST')]['PRICE'],
              'type': 'line',
              'name': val + " - Close Price",
              } for val in dropdown_value]

    data2 = [{'x': df[(df['SECURITY_CODE'] == val) & (df['TYPE'] == 'FORECAST')]['WORK_DATE'],
              'y': df[(df['SECURITY_CODE'] == val) & (df['TYPE'] == 'FORECAST')]['HIGH_PRICE'],
              'type': 'line',
              'name': val + " - High Price",
              } for val in dropdown_value]

    # data3 = [{'x': df[(df['SECURITY_CODE'] == '0050') & (df['TYPE'] == 'PAST')]['WORK_DATE'],
    #           'y': df[(df['SECURITY_CODE'] == '0050') & (df['TYPE'] == 'PAST')]['LOW_PRICE'],
    #           'type': 'line',
    #             'name': val + " - High Price",
    #           } for val in dropdown_value]

    # data4 = [{'x': df[df[1] == val][0],
    #               'y': df[df[1] == val][5],
    #               'type': 'line',
    #               'name': val + " - 平均價",
    #               } for val in dropdown_value]
    #
    # data5 = [{'x': df[df[1] == val][0],
    #               'y': df[df[1] == val][6],
    #               'type': 'line',
    #               'name': val + " - 交易量",
    #               } for val in dropdown_value]

    figure = [
        dcc.Graph(
            id='example-graph',
            figure={
                'data': data1 + data2,
                'layout': {
                    'plot_bgcolor': colors['background'],
                    'paper_bgcolor': colors['background'],
                    'font': {
                        'color': colors['text']
                    },
#                    'title': 'DASH'
                }
            }
        ),
    ]

    return figure



if __name__ == '__main__':
   app.run_server()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET /_dash-component-suites/dash_renderer/react@16.v1_4_1m1592122177.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_4_1m1592122177.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_4_1m1592122177.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_4_1m1592122177.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_10_0m1592122177.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 15:13:01] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_0m1592122177.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2020 1